In [5]:
import pandas as pd
import numpy as np
from imvc.datasets import LoadDataset
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error

ModuleNotFoundError: No module named 'imvc.datasets'

In [ ]:
from imvc.cluster import IMSR

In [ ]:
def dataset_engine_comp(results_dict, Xs, y, n_clusters, estimator, engines, ps, n_times):
    for p in ps:
        results_dict[p] = {}
        matrices_comp = {}
        for engine in engines:
            results_dict[p][engines] = {}
            matrices_comp[engine] = []
            for i in n_times:
                results_dict[p][engines][i] = {}
                estimator.set_params(n_clusters=n_clusters, engine=engine, random_state=i)
                start_time = perf_counter()
                labels = estimator.fit_predict(Xs)
                results_dict[p][engine][i]["Computing time"] = perf_counter() - start_time
                results_dict[p][engine][i]["AMI"] = adjusted_mutual_info_score(labels_true=y, labels_pred=labels)
                results_dict[p][engine][i]["ARI"] = adjusted_rand_score(labels_true=y, labels_pred=labels)
                if hasattr(estimator, "embeddings_"):
                    matrices_comp[engine].append(estimator.embeddings_)
        if hasattr(estimator, "embeddings_"):
            for engine in engines + ["both"]: 
                rmse_list = []
                mae_list = []
                if engine == "both":
                    mats = [mat for mats in matrices_comp.values() for mat in mats]
                else:
                    mats = matrices_comp[engine]
                for mat1, mat2 in set(itertools.combinations(mats, 2)):
                    rmse_list.append(root_mean_squared_error(y_true=mat1, y_pred=mat2))
                    mae_list.append(mean_absolute_error(y_true=mat1, y_pred=mat2))
                results_dict[p][engine]["RMSE"] = np.mean(rmse_list)
                results_dict[p][engine]["MAE"] = np.mean(mae_list)
    return results_dict

In [ ]:
def dataset_engine_comp(datasets, estimator, engines, ps, n_times):
    results = {}
    for dataset in datasets:
        names = dataset_name.split("_")
        if "simulated" in names:
            names = ["_".join(names)]
        x_name,y_name = names if len(names) > 1 else (names[0], "0")
        Xs, y = LoadDataset.load_dataset(dataset_name=x_name, return_y=True)
        y = y[y_name]
        n_clusters = int(y.nunique())
        
        results[dataset] = {}
        results = dataset_engine_comp(results_dict= results[dataset], Xs=Xs, y=y, n_clusters=n_clusters,
                                      estimator=estimator, engines=engines, ps=ps, n_times=n_times)
    results = pd.DataFrame(results)
    return results

In [ ]:
results = dataset_engine_comp(datasets= [], estimator=IMSR, engines= ["python", "matlab"], ps= np.arange(0., 0.7, 0.2), n_times = 50)